1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 답변 생성이 오래 걸림
3. 임베딩 -> 백터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [15]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding, persist_directory="./chroma", collection_name='chroma_tax')
database = Chroma(embedding_function=embedding, persist_directory='./chroma', collection_name='chroma_tax')

In [ ]:
query = "5000만원인 세금 알려줘"
retreived_docs = database.similarity_search(query)

[Document(id='b5d3d5f1-61ed-44ed-b91d-be7050ab85a0', metadata={'source': './tax.docx'}, page_content='④ 관할세무서장 또는 지방국세청장은 주소ㆍ거소 또는 사업장의 이동이 빈번하다고 인정되는 지역의 납세의무가 있는 자에 대해서는 제1항과 제2항을 준용하여 대통령령으로 정하는 바에 따라 수시부과할 수 있다.\n\n⑤ 수시부과 절차와 그 밖에 필요한 사항은 대통령령으로 정한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제83조(과세표준과 세액의 통지) 납세지 관할 세무서장 또는 지방국세청장은 제80조에 따라 거주자의 과세표준과 세액을 결정 또는 경정한 경우에는 그 내용을 해당 거주자 또는 상속인에게 대통령령으로 정하는 바에 따라 서면으로 통지하여야 한다. 다만, 제42조에 따라 과세표준과 세액의 결정 또는 경정을 한 경우에는 지체 없이 통지하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제84조(기타소득의 과세최저한) 기타소득이 다음 각 호의 어느 하나에 해당하면 그 소득에 대한 소득세를 과세하지 아니한다. <개정 2014. 12. 23., 2015. 12. 15., 2022. 12. 31.>\n\n1. 제21조제1항제4호에 따른 환급금으로서 건별로 승마투표권, 승자투표권, 소싸움경기투표권, 체육진흥투표권의 권면에 표시된 금액의 합계액이 10만원 이하이고 다음 각 목의 어느 하나에 해당하는 경우\n\n가. 적중한 개별투표당 환급금이 10만원 이하인 경우\n\n나. 단위투표금액당 환급금이 단위투표금액의 100배 이하이면서 적중한 개별투표당 환급금이 200만원 이하인 경우\n\n2. 제14조제3항제8호라목에 따른 복권 당첨금(복권당첨금을 복권 및 복권 기금법령에 따라 분할하여 지급받는 경우에는 분할하여 지급받는 금액의 합계액을 말한다) 또는 제21조제1항제14호에 따른 당첨금품등이 건별로 200만원 이하인 경우\n\n3. 그 밖의 기타소득금액(제21조제1

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retreived_docs}

Question: {query}
"""

ai_message = llm.invoke(prompt)

In [8]:
print(ai_message)

content='귀하의 질문은 세금과 관련된 금액이나 상황을 구체적으로 설명하지 않기 때문에 정확한 답변을 제공하기 어렵습니다. 일반적으로 한국 소득세를 계산하려면 소득 유형(예: 근로소득, 사업소득, 기타소득 등), 공제 항목(예: 기부금, 보험료 등), 그리고 소득세율을 포함한 다양한 정보가 필요합니다.\n\n5000만원의 소득세 금액을 알려달라는 요청으로 이해하면, 이는 과세표준에 따라 다르며, 공제 항목 및 세율에 따라 달라질 수 있습니다. 예를 들어, 종합소득세율은 과세표준에 따라 6%에서 45%까지 적용되며, 기본공제 및 추가공제를 통해 과세표준이 달라질 수 있습니다.\n\n보다 구체적인 답변을 원하시면 소득의 유형, 과세표준, 적용되는 공제 등을 포함하여 추가 정보를 제공해 주시기 바랍니다. 그러면 보다 정확한 정보를 제공해드리도록 하겠습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 234, 'prompt_tokens': 4152, 'total_tokens': 4386, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-BzdO8FpKAQMQ4VZc5rOFtvqORzTBs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--3ebc83a1-ae38-4d23-a9

In [9]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/kimjunhwan/inflearn-llm/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA

rag_prompt = ChatPromptTemplate.from_messages([
    ('human', '''You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
                Question: {question} 
                Context: {context} 
                Answer:''')
])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": rag_prompt}
)

In [11]:
ai_message = qa_chain({"query": query})

/var/folders/jh/2xkmnd7x5w5_dpbfyzb8y1bw0000gn/T/ipykernel_7812/3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [12]:
print(ai_message)

{'query': '5000만원인 세금 알려줘', 'result': '죄송합니다. 제공된 문맥에서는 5000만원의 세금에 대한 정보를 찾을 수 없습니다. 자세한 세금 계산은 세무 전문가에게 문의하시기 바랍니다.'}
